In [6]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 522.1 kB/s eta 0:00:00m eta 0:00:010:00:02
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)


In [1]:
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, MeanShift, MiniBatchKMeans, Birch, AffinityPropagation, AgglomerativeClustering
from sklearn import metrics

In [456]:
df_data = pd.read_csv("../../results_demo/localization/NLP/prottrans/prottrans_bert_bfd-localization.csv")
df_data

,id,target,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,...,p_1014,p_1015,p_1016,p_1017,p_1018,p_1019,p_1020,p_1021,p_1022,p_1023
0,sequence_51016,-4.626936,0.0136,0.0132,-0.0103,0.0071,0.0485,0.0117,-0.0241,0.0175,...,0.0033,-0.0297,-0.0106,0.0147,0.0638,-0.0409,0.0177,0.0065,-0.0526,-0.0009
1,sequence_51170,-5.599110,0.0222,0.0125,-0.0099,0.0118,0.0571,0.0126,-0.0331,0.0140,...,0.0107,-0.0482,-0.0198,0.0136,0.0563,-0.0370,0.0166,0.0121,-0.0457,0.0085
2,sequence_32130,-5.715788,-0.0053,0.0133,-0.0090,0.0103,0.0602,-0.0045,-0.0350,0.0110,...,0.0077,-0.0451,-0.0244,0.0232,0.0684,-0.0376,0.0221,0.0055,-0.0592,0.0053
3,sequence_51136,-5.335352,0.0145,0.0076,-0.0150,0.0085,0.0512,0.0005,-0.0305,0.0158,...,0.0118,-0.0581,-0.0134,0.0119,0.0677,-0.0320,0.0213,0.0075,-0.0537,0.0105
4,sequence_51191,-4.187052,0.0206,0.0147,-0.0090,0.0124,0.0483,0.0132,-0.0295,0.0199,...,0.0059,-0.0386,-0.0125,0.0113,0.0637,-0.0357,0.0147,0.0046,-0.0523,0.0057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243,sequence_50986,-6.800648,0.0135,0.0074,-0.0092,0.0082,0.0426,0.0157,-0.0275,0.0148,...,0.0054,-0.0285,-0.0139,0.0135,0.0632,-0.0416,0.0155,0.0065,-0.0538,0.0018
244,sequence_51042,-4.418679,0.0031,0.0182,-0.0111,-0.0005,0.0602,0.0073,-0.0235,0.0139,...,0.0067,-0.0356,-0.0209,0.0129,0.0631,-0.0425,0.0119,0.0073,-0.0524,0.0055
245,sequence_51024,-4.606671,0.0041,0.0167,-0.0160,0.0003,0.0554,0.0055,-0.0197,0.0159,...,0.0069,-0.0397,-0.0135,0.0144,0.0612,-0.0419,0.0183,0.0054,-0.0547,0.0047
246,sequence_51037,-4.148785,0.0131,0.0149,-0.0099,0.0043,0.0621,0.0116,-0.0283,0.0093,...,0.0028,-0.0319,-0.0204,0.0114,0.0663,-0.0398,0.0179,0.0103,-0.0500,0.0016


In [457]:
def get_performance_clustering(data, labels):
    siluetas = metrics.silhouette_score(data, labels, metric='euclidean')
    calinski = metrics.calinski_harabasz_score(data, labels)
    davies = metrics.davies_bouldin_score(data, labels)

    return siluetas, calinski, davies

In [458]:
#ignoramos y removemos las columnas que no ocuparemos para el clustering
ignore_columns = pd.DataFrame()
ignore_columns['id'] = df_data['id']
ignore_columns['target'] = df_data['target']

df_data = df_data.drop(columns=['id', 'target'])

In [459]:
#exploramos KNN
matrix_result = []
for k in range(2, 30):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_data)
    siluetas, calinski, davies = get_performance_clustering(df_data, kmeans.labels_)
    row = ["k-means-{}".format(k), siluetas, calinski, davies]
    matrix_result.append(row)

df_explore = pd.DataFrame(matrix_result, columns=['strategy', 'siluetas', 'calinski', 'davies'])

In [460]:
#seleccionamos el mejor
import numpy as np

highest_siluetas = np.max(df_explore['siluetas'])
highest_calinski = np.max(df_explore['calinski'])

# filtramos
df_filter_by_siluetas = df_explore.loc[df_explore['siluetas'] >= highest_siluetas]
df_filter_by_calinski = df_explore.loc[df_explore['calinski'] >= highest_calinski]

df_concat = pd.concat([df_filter_by_siluetas, df_filter_by_calinski])
strategies = df_concat['strategy'].unique()
strategies


array(['k-means-2'], dtype=object)

In [461]:
# en este caso salio el mejor k-means=2
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(df_data)
ignore_columns['labels'] = kmeans.labels_

ignore_columns.to_csv("../../results_demo/localization/NLP/prottrans/unsupervised_clustering_sequences.csv")